In [2]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("mental_health.pdf")
docs = loader.load()
docs

[Document(metadata={'source': 'mental_health.pdf', 'page': 0}, page_content='Mental Health\nis for Everyone\nPromoting Mental Health In Adults\n'),
 Document(metadata={'source': 'mental_health.pdf', 'page': 1}, page_content='Mental Health is for Everyone2This resource is to promote \nmental health.\nFor crisis help:\n•Call 911 \n•Visit your nearest hospital emergency department\nWhat’s Inside\nWhat is mental health and mental illness ............................ 3\nMental health and mental illness diagram .......................... 5\nTransition and life events ........................................................... 6\nMental health and mental illness examples ....................... 7\nProtective and risk factor ........................................................... 10\nWhat supports mental health? ................................................ 12\nThings you can do right now to improve                              your mental health\n.......................................

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(metadata={'source': 'mental_health.pdf', 'page': 0}, page_content='Mental Health\nis for Everyone\nPromoting Mental Health In Adults'),
 Document(metadata={'source': 'mental_health.pdf', 'page': 1}, page_content='Mental Health is for Everyone2This resource is to promote \nmental health.\nFor crisis help:\n•Call 911 \n•Visit your nearest hospital emergency department\nWhat’s Inside\nWhat is mental health and mental illness ............................ 3\nMental health and mental illness diagram .......................... 5\nTransition and life events ........................................................... 6\nMental health and mental illness examples ....................... 7\nProtective and risk factor ........................................................... 10\nWhat supports mental health? ................................................ 12\nThings you can do right now to improve                              your mental health\n.........................................

In [4]:
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'mental_health.pdf', 'page': 0}, page_content='Mental Health\nis for Everyone\nPromoting Mental Health In Adults'),
 Document(metadata={'source': 'mental_health.pdf', 'page': 1}, page_content='Mental Health is for Everyone2This resource is to promote \nmental health.\nFor crisis help:\n•Call 911 \n•Visit your nearest hospital emergency department\nWhat’s Inside\nWhat is mental health and mental illness ............................ 3\nMental health and mental illness diagram .......................... 5\nTransition and life events ........................................................... 6\nMental health and mental illness examples ....................... 7\nProtective and risk factor ........................................................... 10\nWhat supports mental health? ................................................ 12\nThings you can do right now to improve                              your mental health\n.........................................

In [5]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents[:10],OllamaEmbeddings())

In [6]:
db

In [7]:
query="A mental illness is when a person is diagnosed"
result=db.similarity_search(query)
result[0].page_content

'Mental Health is for Everyone3What is mental health?\nGood mental health is important for everyone as it helps us \ncope with life’s stresses and reach our goals. Mental health is about how we think, feel and act. When our mental health is good it helps us to get the most out of life. \nWe all have highs and lows in our mental health. A low period \ndoes not mean you have a mental illness. It’s normal to have feelings like anger, sadness or anxiety when facing challenges.\nWhat is mental illness?\nThere is a difference between mental health and mental illness. A mental illness is when a person is diagnosed with a problem that alters their thinking, mood and/or behaviour. Illnesses are linked to times when a person is distressed and their ability to function well has changed. Examples of mental illness include depression and anxiety disorder. \nWhether or not you have an illness, knowing about mental'

In [8]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama3")
llm

Ollama(model='llama3')

In [9]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
<context>
{context}
</context>
Question: {input}""")

In [10]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [11]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002255B1C67A0>)

In [12]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [13]:
response=retrieval_chain.invoke({"input":"What is Mental Health?"})

In [14]:
response

{'input': 'What is Mental Health',
 'context': [Document(metadata={'source': 'mental_health.pdf', 'page': 2}, page_content='health and how to keep mentally healthy is important for everyone. We can all improve our mental health.'),
  Document(metadata={'source': 'mental_health.pdf', 'page': 8}, page_content='Mental Health is for Everyone9\nBill\nIn his 20’s, Bill was diagnosed \nand treated for depression and recovered. \nBill recently lost his job with a \ncompany he had worked at for the past 25 years. He is angry, having trouble sleeping and drinking heavily. Bill has started to have thoughts of suicide. \nBill remembers from his previous \nepisode of depression that there is help available and decides to call the local crisis line.'),
  Document(metadata={'source': 'mental_health.pdf', 'page': 0}, page_content='Mental Health\nis for Everyone\nPromoting Mental Health In Adults'),
  Document(metadata={'source': 'mental_health.pdf', 'page': 5}, page_content='Mental Health is for Every

In [15]:
response['answer']

"Based on the provided context, Mental Health refers to the overall well-being and emotional stability of an individual. It encompasses the ability to cope with life's transitions and events, manage stress, and maintain a positive outlook. The context suggests that mental health is important for everyone and can be improved with proper care and attention."